# Aula de Visão Computacional

### Vamos baixar um modelo pré treinado - Yolo

In [29]:
#!curl -O https://pjreddie.com/media/files/yolov3.weights
#!curl -O https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
#!curl -O https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names



> yolov3.weights: https://pjreddie.com/media/files/yolov3.weights

> yolov3.cfg: https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

> coco.names: https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.name

### Utilizaremos a biblioteca Cv2 que usa a câmera do notebook.

In [30]:
import cv2
import numpy as np

In [31]:
# conda install opencv -y

In [32]:
# Em caso de erro:
#pip uninstall opencv-python
#conda install -c conda-forge opencv=4.5.3

### Usando DNN - Redes Neurais Profundas

In [33]:
# Carregar os arquivos do modelo YOLO (os arquivos que nós baixamos)
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

In [34]:
# Carregar as classes - e transformando em uma lista
classes = []
f = open('coco.names', 'r')
lines = f.readlines()
f.close()

for line in lines:
    classes.append(line.strip())

In [35]:
# Configurar cores aleatórias para cada classe
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [36]:
# Carregar a imagem de exemplo
image = cv2.imread('exemplo.jpg')

In [37]:
# Redimensionar a imagem se necessário
# image = cv2.resize(image, None, fx=0.4, fy=0.4)

### Criando blob

Um "blob" é essencialmente uma matriz multidimensional que contém os valores de pixel normalizados da imagem, juntamente com informações adicionais, como dimensões, canais e outras configurações específicas da rede neural.

In [38]:
# Obter as dimensões da imagem
height, width, _ = image.shape

# Construir um blob a partir da imagem para o modelo YOLO
blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)

### Usando yolo

In [39]:
# Definir as camadas de saída
#output_layers_names = net.getUnconnectedOutLayersNames()

output_names = net.getUnconnectedOutLayersNames()
output_layers = [net.getLayer(name) for name in output_names]
output_layers = [layer.name for layer in output_layers]

In [40]:
# Passar o blob pela rede neural
net.setInput(blob)
outputs = net.forward(output_layers)

In [ ]:
# Configurar as listas de detecção
class_ids = []
confidences = []
boxes = []

Neste trecho de código, estamos analisando as saídas (outputs) da rede neural, que são as informações sobre os objetos detectados na imagem. Vamos percorrer cada uma dessas saídas para obter as informações de cada objeto detectado.

In [ ]:
# Processar as saídas da rede neural
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5:
            # Obter as coordenadas do objeto detectado
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Calcular os cantos do retângulo delimitador
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            # Adicionar as informações à lista de detecção
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [ ]:
# Aplicar a supressão não máxima para eliminar detecções redundantes
indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

In [ ]:
# Desenhar os retângulos delimitadores e exibir as classes nas detecções
for i in indices:
    box = boxes[i]
    x, y, w, h = box
    label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'

    color = colors[class_ids[i]]
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Exibir a imagem com as detecções
cv2.imshow('Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
